In [22]:
import os
import math
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


In [23]:
BASE_DIR = Path.cwd().parent
DATA_PATH = BASE_DIR / "dataset" / "card_usage" / "card_subway_with_timeseries_features.csv"
OUT_DIR = BASE_DIR / "ml_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DATE_COL = "date"
TARGET_COL = "total_flow"

SEQ_LEN = 14
BATCH_SIZE = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


Device: mps


In [24]:

DATE_COL = "date"
TARGET_COL = "total_flow"

ROOT = Path.cwd().parent   
DATA_PATH = ROOT / "dataset" / "card_usage" / "card_subway_transform_cleaned.csv"

df = pd.read_csv(DATA_PATH, parse_dates=[DATE_COL], low_memory=False)


# numeric cleanup
for col in ["boardings", "alightings", "latitude", "longitude", "station_code", "seoulmetro_code", TARGET_COL]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# weekend -> int
if "is_weekend" in df.columns:
    df["is_weekend"] = pd.to_numeric(df["is_weekend"], errors="coerce").fillna(0).astype(int)
else:
    df["is_weekend"] = (df[DATE_COL].dt.weekday >= 5).astype(int)

# station_key
code = None
if "seoulmetro_code" in df.columns:
    code = df["seoulmetro_code"]
elif "station_code" in df.columns:
    code = df["station_code"]

if code is not None:
    code_int = code.fillna(-1).astype(int)
    df["station_key"] = np.where(
        code.notna() & (code.astype(float) > 0),
        code_int.astype(str),
        df["line"].astype(str) + "|" + df["station_kr"].astype(str)
    )
else:
    df["station_key"] = df["line"].astype(str) + "|" + df["station_kr"].astype(str)

df = df.sort_values(["station_key", DATE_COL]).reset_index(drop=True)

In [25]:
# Feature engineering
df["day_of_week_num"] = df[DATE_COL].dt.dayofweek
df["day_of_month"] = df[DATE_COL].dt.day
df["week_of_year"] = df[DATE_COL].dt.isocalendar().week.astype(int)
df["flow_ratio"] = df["boardings"] / (df["alightings"] + 1)
df["flow_diff"] = df["boardings"] - df["alightings"]

LAGS = [1, 7, 14]
for lag in LAGS:
    df[f"flow_lag_{lag}"] = df.groupby("station_key")[TARGET_COL].shift(lag)

ROLL_WINDOWS = [7, 14]
for w in ROLL_WINDOWS:
    df[f"flow_roll_mean_{w}"] = df.groupby("station_key")[TARGET_COL].transform(
        lambda s: s.shift(1).rolling(w, min_periods=w).mean()
    )
    df[f"flow_roll_std_{w}"] = df.groupby("station_key")[TARGET_COL].transform(
        lambda s: s.shift(1).rolling(w, min_periods=w).std()
    )

# Target: next day flow
df["target"] = df.groupby("station_key")[TARGET_COL].shift(-1)

FEATURES = [
    "flow_lag_1", "flow_lag_7", "flow_lag_14",
    "flow_roll_mean_7", "flow_roll_mean_14",
    "flow_roll_std_7", "flow_roll_std_14",
    "flow_ratio", "flow_diff",
    "day_of_week_num", "day_of_month", "week_of_year",
    "is_weekend"
]
for g in ["latitude", "longitude"]:
    if g in df.columns:
        FEATURES.append(g)

needed = FEATURES + ["target", DATE_COL, "station_key"]
df_model = df.dropna(subset=needed).reset_index(drop=True)


In [29]:
# Time split
split_date = df_model[DATE_COL].quantile(0.8)
train_df = df_model[df_model[DATE_COL] <= split_date].copy()
val_df = df_model[df_model[DATE_COL] > split_date].copy()

# Per-station normalization
stats = train_df.groupby("station_key")[TARGET_COL].agg(["mean", "std"]).reset_index()
stats["std"] = stats["std"].replace(0, np.nan)
stats = stats.fillna({"std": 1.0})

train_df = train_df.merge(stats, on="station_key", how="left")
val_df = val_df.merge(stats, on="station_key", how="left")

train_df["flow_norm"] = (train_df[TARGET_COL] - train_df["mean"]) / train_df["std"]
val_df["flow_norm"] = (val_df[TARGET_COL] - val_df["mean"]) / val_df["std"]

val_df["mean"] = val_df["mean"].fillna(val_df[TARGET_COL].mean())
val_df["std"] = val_df["std"].fillna(val_df[TARGET_COL].std() if val_df[TARGET_COL].std() > 0 else 1.0)
val_df["flow_norm"] = (val_df[TARGET_COL] - val_df["mean"]) / val_df["std"]

# --- Filter out rows with NaN flow_lag_1 to match LGBM baseline ---
val_df = val_df[val_df["flow_lag_1"].notna()].reset_index(drop=True)

# Calculate and print baseline BEFORE LSTM (on all valid val_df rows)
baseline_mae = np.mean(np.abs(val_df["target"] - val_df["flow_lag_1"]))
baseline_rmse = np.sqrt(np.mean((val_df["target"] - val_df["flow_lag_1"]) ** 2))
print("Baseline (flow_lag_1) BEFORE LSTM:")
print(f"MAE  : {baseline_mae:,.2f}")
print(f"RMSE : {baseline_rmse:,.2f}")
print("-"*40)

# Build sequences per station
possible_features = [
    "flow_norm", "is_weekend", "day_of_week_num", "day_of_month", "week_of_year",
    "flow_lag_1", "flow_lag_7", "flow_lag_14", "flow_roll_mean_7", "flow_roll_mean_14",
    "flow_roll_std_7", "flow_roll_std_14", "flow_ratio", "flow_diff"
]
if "latitude" in df.columns and "longitude" in df.columns:
    possible_features += ["latitude", "longitude"]

FEATURE_COLS = [col for col in possible_features if col in df.columns]
print("Using features:", FEATURE_COLS)

Baseline (flow_lag_1) BEFORE LSTM:
MAE  : 11,342.17
RMSE : 23,829.96
----------------------------------------
Using features: ['is_weekend', 'day_of_week_num', 'day_of_month', 'week_of_year', 'flow_lag_1', 'flow_lag_7', 'flow_lag_14', 'flow_roll_mean_7', 'flow_roll_mean_14', 'flow_roll_std_7', 'flow_roll_std_14', 'flow_ratio', 'flow_diff', 'latitude', 'longitude']


In [ ]:
class SubwaySeqDataset(Dataset):
    """
    Each item:
      X: (SEQ_LEN, F) features for days t-SEQ_LEN+1..t
      y: scalar flow_norm at day t+1
      meta: station_key, date_of_y, mean, std (for denorm)
    """
    def __init__(self, frame: pd.DataFrame, seq_len: int, feature_cols: list[str]):
        self.seq_len = seq_len
        self.feature_cols = feature_cols

        self.samples = []
        for st, g in frame.groupby("station_key"):
            g = g.sort_values(DATE_COL).reset_index(drop=True)
            if len(g) < seq_len + 1:
                continue

            feats = g[feature_cols].to_numpy(dtype=np.float32)
            y = g["flow_norm"].to_numpy(dtype=np.float32)
            mu = g["mean"].to_numpy(dtype=np.float32)
            sd = g["std"].to_numpy(dtype=np.float32)
            dates = g[DATE_COL].dt.strftime("%Y-%m-%d").astype(str).to_numpy()

            for t in range(seq_len - 1, len(g) - 1):
                X = feats[t - (seq_len - 1): t + 1]
                target = y[t + 1]
                meta = {
                    "station_key": str(st),
                    "date_y": dates[t + 1],
                    "mean": float(mu[t + 1]),
                    "std": float(sd[t + 1]),
                }
                self.samples.append((X, target, meta))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        X, y, meta = self.samples[idx]
        return torch.from_numpy(X), torch.tensor(y, dtype=torch.float32), meta

train_ds = SubwaySeqDataset(train_df, SEQ_LEN, FEATURE_COLS)
val_ds = SubwaySeqDataset(val_df, SEQ_LEN, FEATURE_COLS)

print("Train samples:", len(train_ds))
print("Val samples  :", len(val_ds))

def custom_collate(batch):
    Xs, ys, metas = zip(*batch)
    return torch.stack(Xs), torch.stack(ys), list(metas)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=custom_collate)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate)



Train samples: 2238141
Val samples  : 553566


In [ ]:
# Model: LSTM
class LSTMRegressor(nn.Module):
    def __init__(self, input_size: int, hidden_size: int = 128, num_layers: int = 2, dropout: float = 0.0):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.head(last).squeeze(-1)

model = LSTMRegressor(input_size=len(FEATURE_COLS)).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

def eval_loader(loader):
    model.eval()
    preds_norm, trues_norm, metas = [], [], []
    with torch.no_grad():
        for X, y, meta in loader:
            X = X.to(DEVICE, dtype=torch.float32)
            y = y.to(DEVICE, dtype=torch.float32)
            yhat = model(X)
            preds_norm.append(yhat.detach().cpu().numpy())
            trues_norm.append(y.detach().cpu().numpy())
            if isinstance(meta, dict):
                metas.append(meta)
            else:
                metas.extend(list(meta))
    preds_norm = np.concatenate(preds_norm)
    trues_norm = np.concatenate(trues_norm)
    mu = np.array([m["mean"] for m in metas], dtype=np.float32)
    sd = np.array([m["std"] for m in metas], dtype=np.float32)
    preds = preds_norm * sd + mu
    trues = trues_norm * sd + mu
    mae = float(np.mean(np.abs(preds - trues)))
    rmse = float(math.sqrt(np.mean((preds - trues) ** 2)))
    return mae, rmse, preds, trues, metas

history = {"train_loss": [], "val_mae": [], "val_rmse": []}

for epoch in range(1, EPOCHS + 1):
    model.train()
    running = 0.0
    for X, y, _ in train_loader:
        X = X.to(DEVICE, dtype=torch.float32)
        y = y.to(DEVICE, dtype=torch.float32)
        optimizer.zero_grad()
        yhat = model(X)
        loss = criterion(yhat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running += float(loss.item())
    train_loss = running / max(1, len(train_loader))
    val_mae, val_rmse, _, _, _ = eval_loader(val_loader)
    history["train_loss"].append(train_loss)
    history["val_mae"].append(val_mae)
    history["val_rmse"].append(val_rmse)
    print(f"Epoch {epoch}/{EPOCHS} | train_loss={train_loss:.5f} | val_MAE={val_mae:,.2f} | val_RMSE={val_rmse:,.2f}")

val_mae, val_rmse, preds, trues, metas = eval_loader(val_loader)

print("\nLSTM Results (Validation Set)")
print(f"MAE  : {val_mae:,.2f}")
print(f"RMSE : {val_rmse:,.2f}")
print("-"*40)
print("Improvement over baseline")
print(f"MAE improvement  : {(baseline_mae - val_mae) / baseline_mae * 100:.2f}%")
print(f"RMSE improvement : {(baseline_rmse - val_rmse) / baseline_rmse * 100:.2f}%")

Epoch 1/20 | train_loss=0.74701 | val_MAE=8,510.34 | val_RMSE=16,538.67
Epoch 2/20 | train_loss=0.70634 | val_MAE=8,420.25 | val_RMSE=16,407.88


KeyboardInterrupt: 